<a href="https://colab.research.google.com/github/anantrp/ai-experiments/blob/main/openai_function_calling_stock_prices_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
!pip install openai
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
global OPENAI_API_KEY
global POLYGON_API_KEY

OPENAI_API_KEY=""
POLYGON_API_KEY=""

openai.api_key = OPENAI_API_KEY

In [133]:
import datetime
import requests
import json
import openai

global today_str

today = datetime.date.today()
today_str = today.strftime('%Y-%m-%d')

def get_stock_price_by_dates(ticker_symbol, date):
    url = f"https://api.polygon.io/v1/open-close/{ticker_symbol}/{date}?adjusted=true&apiKey={POLYGON_API_KEY}"
    try:
        response = requests.request(method='GET', url=url)
        response.raise_for_status()
        # print(response.text)
        return json.dumps(response.text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [140]:
def run_conversation(message):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"Date today is {today_str}. Provide answer with a date. {message}"}],
        functions=[
            {
                "name": "get_stock_price_by_dates",
                "description": "Get stock prices by dates",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker_symbol": {
                            "type": "string",
                            "description": "The ticker symbol for the stock listed on NASDAQ exchange",
                        },
                        "date": {
                            "type": "string",
                            "description": "date string in %Y-%m-%d format"
                        }
                    },
                    "required": ["stock_ticker"],
                },
            }
        ],
        function_call="auto",
    )

    internal_message = response["choices"][0]["message"]
    # print(internal_message)

    if internal_message.get("function_call"):
        function_name = internal_message["function_call"]["name"]
        function_args = json.loads(internal_message["function_call"]["arguments"])

        function_response = get_stock_price_by_dates(
            ticker_symbol=function_args.get("ticker_symbol"),
            date=function_args.get("date"),
        )

        # print(function_response)
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": message},
                internal_message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

In [141]:
print(run_conversation("tell me apple volume 3 days ago")["choices"][0]["message"]["content"])

Apple's volume 3 days ago (on June 13, 2023) was approximately 54,867,129 shares.
